# Distributed Machine Learning using Tensorflow + Cloud ML Engine

A barebones example showing the simplest path from raw data to distributed cloud training with the fewest lines of code

This notebook is intended to be run on Google Cloud Datalab: https://cloud.google.com/datalab/docs/quickstarts


In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
print(tf.__version__)

1.0.0


## Steps

1. Load raw data

2. Define feature columns

3. Define Estimator

4. Define input function

5. Define Experiment

6. Package Code

7. Run!

### 1) Load Raw Data

The boston housing prices dataset. It is a tab separated text file hosted in a public GCS bucket.


In [3]:
#downlad data from GCS and store as pandas dataframe 
df = pd.read_csv(
  filepath_or_buffer='https://storage.googleapis.com/vijay-public/boston_housing/housing.data.txt',
  delim_whitespace=True,
  names=["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT","MEDV"])

In [4]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


### 2) Define feature columns

Feature columns are your Estimator's data "interface." They tell the estimator in what format they should expect data and how to interpret it (is it one-hot? sparse? dense? categorical? conteninous?).  https://www.tensorflow.org/api_docs/python/tf/contrib/layers/feature_column




In [5]:
FEATURES = ["CRIM", "ZN", "INDUS", "NOX", "RM",
            "AGE", "DIS", "TAX", "PTRATIO"]
LABEL = "MEDV"

feature_cols = [tf.contrib.layers.real_valued_column(k)
                  for k in FEATURES] #list of Feature Columns

### 3) An Estimator is what actually implements your training, eval and prediction loops. Every estimator has the following methods:

- fit() for training
- eval() for evaluation
- predict() for prediction
- export_savedmodel() for writing model state to disk

Tensorflow has several canned estimator that already implement these methods (DNNClassifier, LogisticClassifier etc..) or you can implement a custom estimator that subscribes to the tf.contrib.learn.BaseEstimator interface: https://www.tensorflow.org/api_docs/python/tf/contrib/learn/BaseEstimator

For simplicity we will use a canned estimator. To instantiate an estimator simply pass it what Feature Columns to expect.

In [6]:
def generate_estimator(output_dir):
  return tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
                                            hidden_units=[10, 10],
                                            model_dir=output_dir)

### 4) Define input function

Now that you have an estimator and it konws what type of data to expect and how to intepret, you need to actually pass the data to it! This is the job of the input function. 

The input function return a (features, label) tuple
- features: A dictionary. Each key is a feature column name and its value is the Tensor containing the data for that Feature
- label: A Tensor containing the label column

In [7]:
def generate_input_fn(data_set):
    def input_fn():
      features = {k: tf.constant(data_set[k].values) for k in FEATURES}
      labels = tf.constant(data_set[LABEL].values)
      return features, labels
    return input_fn

### 5) Define Experiment

An experiment is ultimatley what you pass to learn_runner.run() for Cloud ML training. It encapsulated the Estimator (which in turn encapsulates the feature column definitions) and the input function.

In [8]:
def generate_experiment_fn(output_dir):
  return tf.contrib.learn.Experiment(
    generate_estimator(output_dir),
    train_input_fn=generate_input_fn(df), 
    eval_input_fn=generate_input_fn(df), #Normally you would pass a different eval set
    train_steps=1000,
    eval_steps=1000
  )
  

### 6) Package Code

You've now written all the tensoflow code you need!

To make it compatible with cloud ML we'll combine the above tensorflow code into a single python file with two simple changes

1. Add some boilerplate code to parse the command line arguments required for gcloud.
2. Use the learn_runner.run() function to run the experiment

We also add an \__init__\.py file to the foler. This is just python convention for identifying modules

In [9]:
%%bash
mkdir trainer
touch trainer/__init__.py

In [10]:
%%writefile trainer/task.py

import argparse
import pandas as pd
import tensorflow as tf
from tensorflow.contrib.learn.python.learn import learn_runner

df = pd.read_csv(
  filepath_or_buffer='https://storage.googleapis.com/vijay-public/boston_housing/housing.data.txt',
  delim_whitespace=True,
  names=["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT","MEDV"])

FEATURES = ["CRIM", "ZN", "INDUS", "NOX", "RM",
            "AGE", "DIS", "TAX", "PTRATIO"]
LABEL = "MEDV"

feature_cols = [tf.contrib.layers.real_valued_column(k)
                  for k in FEATURES] #list of Feature Columns

def generate_estimator(output_dir):
  return tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
                                            hidden_units=[10, 10],
                                            model_dir=output_dir)

def generate_input_fn(data_set):
    def input_fn():
      features = {k: tf.constant(data_set[k].values) for k in FEATURES}
      labels = tf.constant(data_set[LABEL].values)
      return features, labels
    return input_fn

def generate_experiment_fn(output_dir):
  return tf.contrib.learn.Experiment(
    generate_estimator(output_dir),
    train_input_fn=generate_input_fn(df),
    eval_input_fn=generate_input_fn(df),
    train_steps=1000,
    eval_steps=1000
  )

######CLOUD ML ENGINE BOILERPLATE CODE BELOW######
if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  # Input Arguments
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
      required=True
  )
  parser.add_argument(
        '--job-dir',
        help='this model ignores this field, but it is required by gcloud',
        default='junk'
    )
  args = parser.parse_args()
  arguments = args.__dict__
  output_dir = arguments.pop('output_dir')

  learn_runner.run(generate_experiment_fn, output_dir)

Writing trainer/task.py


### 7) Run using gcloud command line tool
Now that our code is packaged we can invoke it using gcloud to run the training. 

Note: Since our dataset is so small and our model is simple the overhead of provisioning the cluster is longer than the actual training time. Accordingly you'll notice the single VM cloud training takes longer than the local training, and the distributed cloud training takes longer than single VM cloud. For larger datasets and more complex models this will reverse

#### Run local
It's a best practice to first run locally on a small dataset to check for errors

In [ ]:
%%bash
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=trainer \
   -- \
   --output_dir='./output'

#### Run on cloud (1 cloud ML unit)

First we specify which GCP project to use. Update the cell below to use your project name.

In [16]:
%%bash
gcloud config set project vijays-sandbox #CHANGE THIS TO YOUR PROJECT

Updated property [core/project].


Then we specify which GCS bucket to write to and a job name.
Job names submitted to the ml engine must be project unique, so we append the system date/time. Update the cell below to point to a GCS bucket you own.

In [17]:
%%bash
GCS_BUCKET=gs://vijays-sandbox-ml/housing #CHANGE THIS TO YOUR GCS BUCKET
JOBNAME=housing_$(date -u +%y%m%d_%H%M%S)

gcloud ml-engine jobs submit training $JOBNAME \
   --region=us-central1 \
   --module-name=trainer.task \
   --package-path=./trainer \
   --job-dir=$GCS_BUCKET/$JOBNAME/ \
   -- \
   --output_dir=$GCS_BUCKET/$JOBNAME/output


jobId: housing_170326_233617
state: QUEUED


Job [housing_170326_233617] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe housing_170326_233617

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs housing_170326_233617


#### Run on cloud (10 cloud ML units)
Because we are using the TF Experiments interface, distributed computing just works! The only change we need to make to run in a distributed fashion is to add the [--scale-tier](https://cloud.google.com/ml/pricing#ml_training_units_by_scale_tier) argument. Cloud ML Engine then takes care of distributing the training across devices for you!


In [18]:
%%bash
GCS_BUCKET=gs://vijays-sandbox-ml/housing #CHANGE THIS TO YOUR GCS BUCKET
JOBNAME=housing_$(date -u +%y%m%d_%H%M%S)

gcloud ml-engine jobs submit training $JOBNAME \
   --region=us-central1 \
   --module-name=trainer.task \
   --package-path=./trainer \
   --job-dir=$GCS_BUCKET/$JOBNAME \
   --scale-tier=STANDARD_1 \
   -- \
   --output_dir=$GCS_BUCKET/$JOBNAME/output

jobId: housing_170326_233648
state: QUEUED


Job [housing_170326_233648] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe housing_170326_233648

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs housing_170326_233648


#### Run on cloud GPU

Also works with GPUs!

"BASIC_GPU" corresponds to one Tesla K80 at the time of this writing, hardware subject to change. It is charged as 3 Cloud ML units. Some types of trainings are better suited for GPU than others. Make sure it speeds up your training at least 3x to be cost effective!

If you want to train across multiple GPUs you would use a [custom scale tier](https://cloud.google.com/ml/docs/concepts/training-overview#job_configuration_parameters)

In [19]:
%%bash
GCS_BUCKET=gs://vijays-sandbox-ml/housing #CHANGE THIS TO YOUR GCS BUCKET
JOBNAME=housing_$(date -u +%y%m%d_%H%M%S)

gcloud ml-engine jobs submit training $JOBNAME \
   --region=us-east1 \
   --module-name=trainer.task \
   --package-path=./trainer \
   --job-dir=$GCS_BUCKET/$JOBNAME \
   --scale-tier=BASIC_GPU \
   -- \
   --output_dir=$GCS_BUCKET/$JOBNAME/output

jobId: housing_170326_233920
state: QUEUED


Job [housing_170326_233920] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe housing_170326_233920

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs housing_170326_233920
